In [1]:
import numpy as np
import pandas as pd
import logging
import os
import glob
import regex as re
import torch
import argparse
import random
import itertools
import random
import ast
import sys
import ast

from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import simpletransformers
from simpletransformers.t5 import T5Model, T5Args
from datasets import load_dataset, get_dataset_split_names

/home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['TORCH_HOME'] = '/scratch/wadhwa.s/cache/'
os.environ['HF_HOME'] = '/scratch/wadhwa.s/cache'
os.environ['TRANSFORMERS_CACHE'] = '/scratch/wadhwa.s/cache'

In [3]:
dataset = load_dataset('ade_corpus_v2', 'Ade_corpus_v2_drug_ade_relation')
ade_dict = {
    "text" : dataset['train']['text'],
    "drug" : dataset['train']['drug'],
    "effect": dataset['train']['effect'],
    "index" : dataset['train']['indexes']
}
df = pd.DataFrame(ade_dict, index=None)
df.shape

Found cached dataset ade_corpus_v2 (/home/wadhwa.s/.cache/huggingface/datasets/ade_corpus_v2/Ade_corpus_v2_drug_ade_relation/1.0.0/940d61334dbfac6b01ac5d00286a2122608b8dc79706ee7e9206a1edb172c559)
100%|███████████████████████████████████████████████████████| 1/1 [00:00<00:00, 414.13it/s]


(6821, 4)

In [4]:
input_docs = "ade_gpt3.txt"

In [5]:
with open(input_docs) as f:
    lines = f.readlines()
    unique_ade = [line.strip() for line in lines]

In [6]:
few_shot = random.sample(unique_ade, 7)

In [7]:
prompt = ""
for i in few_shot:
    de_pairs = []
    for ix, row in df.loc[df['text'] == i].iterrows():
        de_pairs.append([row["drug"], row["effect"]])
    prompt += i + "\n" + str(de_pairs) + "\n\n"
print (prompt)

A 22-year-old black man developed fever, chills, fatigue, night sweats, tender lymphadenopathy, and a generalized, pruritic, macular eruption 3 weeks after starting minocycline therapy for acne.
[['minocycline', 'chills'], ['minocycline', 'fatigue'], ['minocycline', 'fever'], ['minocycline', 'generalized, pruritic, macular eruption'], ['minocycline', 'night sweats']]

Neurological improvement and rehabilitation potential following toxic myelopathy due to intrathecal injection of doxorubicin.
[['doxorubicin', 'toxic myelopathy']]

The cause is presumed to be secondary to hypercoagulability due to asparaginase-induced antithrombin III deficiency.
[['asparaginase', 'antithrombin III deficiency']]

Two patients with imminent gangrene of the extremities caused by ergot-induced arteriospasm underwent mechanical dilatation of the arteries when conventional measures such as anticoagulation, vasodilation, and sympathetic blockade produced no improvement.
[['ergot', 'arteriospasm'], ['ergot', 'g

In [8]:
# with open("few_7_shot_prompt.txt", "w") as text_file:
#     text_file.write(prompt)

In [9]:
with open("few_7_shot_prompt_ade.txt", "r") as text_file:
    prompt = text_file.read()
print (prompt)

Gentamicin-associated acute renal failure.
[['Gentamicin', 'acute renal failure']]</s>

Ampicillin-associated seizures.
[['Ampicillin', 'seizures']]</s>

Approximately 15 min after the first administration of nebulised morphine the patient became markedly bradypneic (respiratory rate: 4-5 bpm), hypotensive (BP 70/40 mmHg), and responded only partially to command.
[['morphine', 'bradypneic'], ['morphine', 'hypotensive']]</s>

Disseminated eruptive giant mollusca contagiosa in an adult psoriasis patient during efalizumab therapy.
[['efalizumab', 'Disseminated eruptive giant mollusca contagiosa']]</s>

Three patients are reported without a history of angina pectoris who had clinical and electrocardiographic evidence of myocardial ischemia during and immediately after BCNU infusion.
[['BCNU', 'myocardial ischemia']]</s>

Vogt-Koyanagi-Harada disease occurring during interferon alpha therapy for chronic hepatitis C.
[['interferon alpha', 'Vogt-Koyanagi-Harada disease']]</s>

Complications o

In [10]:
prompt = prompt + "Temporary neurologic abnormalities were observed in one out of 23 patients undergoing chemotherapy with high-dose methotrexate (HD-MTX) for osteogenic sarcoma."

In [11]:
print (prompt)

Gentamicin-associated acute renal failure.
[['Gentamicin', 'acute renal failure']]</s>

Ampicillin-associated seizures.
[['Ampicillin', 'seizures']]</s>

Approximately 15 min after the first administration of nebulised morphine the patient became markedly bradypneic (respiratory rate: 4-5 bpm), hypotensive (BP 70/40 mmHg), and responded only partially to command.
[['morphine', 'bradypneic'], ['morphine', 'hypotensive']]</s>

Disseminated eruptive giant mollusca contagiosa in an adult psoriasis patient during efalizumab therapy.
[['efalizumab', 'Disseminated eruptive giant mollusca contagiosa']]</s>

Three patients are reported without a history of angina pectoris who had clinical and electrocardiographic evidence of myocardial ischemia during and immediately after BCNU infusion.
[['BCNU', 'myocardial ischemia']]</s>

Vogt-Koyanagi-Harada disease occurring during interferon alpha therapy for chronic hepatitis C.
[['interferon alpha', 'Vogt-Koyanagi-Harada disease']]</s>

Complications o

In [12]:
flan_ade = [prompt + s for s in lines]

In [13]:
print (flan_ade[0])

Gentamicin-associated acute renal failure.
[['Gentamicin', 'acute renal failure']]</s>

Ampicillin-associated seizures.
[['Ampicillin', 'seizures']]</s>

Approximately 15 min after the first administration of nebulised morphine the patient became markedly bradypneic (respiratory rate: 4-5 bpm), hypotensive (BP 70/40 mmHg), and responded only partially to command.
[['morphine', 'bradypneic'], ['morphine', 'hypotensive']]</s>

Disseminated eruptive giant mollusca contagiosa in an adult psoriasis patient during efalizumab therapy.
[['efalizumab', 'Disseminated eruptive giant mollusca contagiosa']]</s>

Three patients are reported without a history of angina pectoris who had clinical and electrocardiographic evidence of myocardial ischemia during and immediately after BCNU infusion.
[['BCNU', 'myocardial ischemia']]</s>

Vogt-Koyanagi-Harada disease occurring during interferon alpha therapy for chronic hepatitis C.
[['interferon alpha', 'Vogt-Koyanagi-Harada disease']]</s>

Complications o

In [14]:
dev_docs = "ade_dev.txt"

In [15]:
with open(dev_docs) as f:
    lines_dev = f.readlines()
    unique_ade_dev = [line.strip() for line in lines_dev]

In [16]:
flan_ade = [prompt + s.strip() for s in lines_dev]

In [17]:
print (prompt)

Gentamicin-associated acute renal failure.
[['Gentamicin', 'acute renal failure']]</s>

Ampicillin-associated seizures.
[['Ampicillin', 'seizures']]</s>

Approximately 15 min after the first administration of nebulised morphine the patient became markedly bradypneic (respiratory rate: 4-5 bpm), hypotensive (BP 70/40 mmHg), and responded only partially to command.
[['morphine', 'bradypneic'], ['morphine', 'hypotensive']]</s>

Disseminated eruptive giant mollusca contagiosa in an adult psoriasis patient during efalizumab therapy.
[['efalizumab', 'Disseminated eruptive giant mollusca contagiosa']]</s>

Three patients are reported without a history of angina pectoris who had clinical and electrocardiographic evidence of myocardial ischemia during and immediately after BCNU infusion.
[['BCNU', 'myocardial ischemia']]</s>

Vogt-Koyanagi-Harada disease occurring during interferon alpha therapy for chronic hepatitis C.
[['interferon alpha', 'Vogt-Koyanagi-Harada disease']]</s>

Complications o

In [6]:
model_args = {
    "cache_dir" : "/scratch/wadhwa.s/cache/",
    "device_map" : "auto",
    "use_multiprocessing": False,
    "use_multiprocessed_decoding": False,
#         "num_train_epochs" : 4,
         "no_save" : True,
#         "preprocess_inputs" : False,
         "overwrite_output_dir" : True,
    "max_seq_length" : 512,
#         "special_tokens_list" : ["<bos>", "<eos>", "<rel>", "<ent>"],
         "max_length" : 100,
#         "num_beams" : 5,
#         "learning_rate": lr,

    }

In [7]:
model = T5Model("t5", "google/flan-t5-xxl", args=model_args)

In [12]:
to_predict = flan_ade[:20]

In [23]:
for i in to_predict:
    de_pairs = []
    text = i.split("\n")[-1]
    print ("INPUT: ", text)
    for ix, row in df.loc[df['text'] == text].iterrows():
        de_pairs.append([row["drug"], row["effect"]])
    print ("TRUE: ", de_pairs)
    print ("GENERATED: ", model.predict([i])[0])
    print ("\n--------------\n")


# for s in to_predict:
#     print (s.split("\n")[-1])
#     print ("\n--------------\n")

INPUT:  These case reports provide evidence that 5-aminosalicylic acid may induce acute pancreatitis after long term treatment.
TRUE:  [['5-aminosalicylic acid', 'acute pancreatitis']]


Generating outputs: 100%|█████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


GENERATED:  [['5-aminosalicylic acid', 'acute pancreatitis']]

--------------

INPUT:  A search of the United States Food and Drug Administration's Adverse Event Reporting System identified nine cases of oxcarbazepine-associated angioedema in pediatric patients aged 16 years and younger.
TRUE:  [['oxcarbazepine', 'angioedema']]


Generating outputs: 100%|█████████████████████████| 1/1 [00:01<00:00,  1.91s/it]


GENERATED:  [['oxcarbazepine', 'angioedema']]

--------------

INPUT:  Human insulin-induced lipoatrophy.
TRUE:  [['Human insulin', 'lipoatrophy']]


...

GENERATED:  [['Scopolamine', 'Anisocoria']]

--------------

INPUT:  Captopril is known to be associated with dermatologic, hematologic, and pulmonary adverse effects.
TRUE:  [['Captopril', 'dermatologic, hematologic, and pulmonary adverse effects']]


Generating outputs: 100%|█████████████████████████| 1/1 [00:01<00:00,  1.56s/it]


GENERATED:  [['Captopril', 'dermatologic']]

--------------

INPUT:  Schneiderian first-rank symptoms associated with fluvoxamine treatment: a case report.
TRUE:  [['fluvoxamine', 'Schneiderian first-rank symptoms']]


Generating outputs: 100%|█████████████████████████| 1/1 [00:02<00:00,  2.01s/it]

GENERATED:  [['fluvoxamine', 'Schneiderian first-rank symptoms']]

--------------



In [15]:
for s in to_predict:
    print (model.predict([s]))

Generating outputs:   0%|                                 | 0/1 [00:00<?, ?it/s]/home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3672: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
Generating outputs: 100%|███

["[['5-aminosalicylic acid', 'acute pancreatitis']]"]


Generating outputs: 100%|█████████████████████████| 1/1 [00:01<00:00,  1.85s/it]


["[['oxcarbazepine', 'angioedema']]"]


Generating outputs: 100%|█████████████████████████| 1/1 [00:01<00:00,  1.49s/it]


...

["[['Scopolamine', 'Anisocoria']]"]


Generating outputs: 100%|█████████████████████████| 1/1 [00:01<00:00,  1.57s/it]


["[['Captopril', 'dermatologic']]"]


Generating outputs: 100%|█████████████████████████| 1/1 [00:02<00:00,  2.02s/it]

["[['fluvoxamine', 'Schneiderian first-rank symptoms']]"]


In [51]:
preds = model.predict(to_predict)

Generating outputs:   0%|                                 | 0/3 [00:04<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_7114/2060853665.py:1 in <cell line: 1>                                            │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_7114/2060853665.py'                         │
│                                                                                                  │
│ /home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/simpletransformers/t5/t5_model.py:10 │
│ 47 in predict                                                                                    │
│                                                                                                  │
│   1044 │   │   │   input_ids = input_ids.to(self.device)                                         │
│   1045 │   │   │   attention_mask = attention_mask.to(self.device)                               │
│   1046 │   │   │                                                                                 │
│ ❱ 1047 │   │   │   outputs = self.model.generate(                                                │
│   1048 │   │   │   │   input_ids=input_ids,                                                      │
│   1049 │   │   │   │   attention_mask=attention_mask,                                            │
│   1050 │   │   │   │   num_beams=self.args.num_beams,                                            │
│                                                                                                  │
│ /home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/torch/autograd/grad_mode.py:27 in    │
│ decorate_context                                                                                 │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:149 │
│ 0 in generate                                                                                    │
│                                                                                                  │
│   1487 │   │   │   │   )                                                                         │
│   1488 │   │   │                                                                                 │
│   1489 │   │   │   # 10. run greedy search                                                       │
│ ❱ 1490 │   │   │   return self.greedy_search(                                                    │
│   1491 │   │   │   │   input_ids,                                                                │
│   1492 │   │   │   │   logits_processor=logits_processor,                                        │
│   1493 │   │   │   │   stopping_criteria=stopping_criteria,                                      │
│                                                                                                  │
│ /home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:223 │
│ 3 in greedy_search                                         

In [43]:
preds[0]

"[['oxcarbazepine', 'angioedema']]"

In [44]:
print (lines_dev[1])

A search of the United States Food and Drug Administration's Adverse Event Reporting System identified nine cases of oxcarbazepine-associated angioedema in pediatric patients aged 16 years and younger.

